In [2]:
%matplotlib inline
import torch
from torch import nn
import numpy as np
from torch.utils import data
from io import StringIO
import random
from d2l import torch as d2l
from matplotlib import pyplot as plt
import torchvision
import re
import os
import glob

In [3]:
# 1. get training data file dir
def get_nonsymm_structdir(nonsymm_struct_dir): 
    fname_non_structure = glob.glob(os.path.join(nonsymm_struct_dir,'*.txt'))
    fname_non_structure.sort(key = lambda 
                             x:(int(re.split('[.]',re.split('_',x)[4])[0])))
    
    return fname_non_structure

def get_symm_structdir(symm_struct_dir): 
    fname_structure = glob.glob(os.path.join(symm_struct_dir,'*.txt'))
    fname_structure.sort(key = lambda 
                             x:(int(re.split('[.]',re.split('_',x)[3])[0])))
    return fname_structure
    
def get_nonsymm_modulidir(nonsymm_moduli_dir): 
    fname_non_moduli = glob.glob(os.path.join(nonsymm_moduli_dir,'*.txt'))
    fname_non_moduli.sort(key = lambda x:(int(re.split('_',x)[3])))
    return fname_non_moduli

def get_symm_modulidir(symm_moduli_dir): 
    fname_moduli = glob.glob(os.path.join(symm_moduli_dir,'*.txt'))
    fname_moduli.sort(key = lambda x:(int(re.split('_',x)[2])))
    return fname_moduli


In [4]:
# 2. read nonsymm training dataset
def read_nonsymm_data(fname_non_structure, fname_non_moduli):
    
    elems_nonsymm  = np.empty(shape = (0, 56))
    elems_nonsymm_size  = np.empty(shape = (1, 56))
    elems  = np.empty(shape = (0, 56))
    elems_non  = np.empty(shape = (0, 56))
    moduli_non = np.empty(shape = (0, 3))
    strut_nonsymm_size = [6, 6, 6]
    strut_size = [6, 6, 6]
    
    for i in range (len(fname_non_structure)):
        fstruture_non_dir = fname_non_structure[i]
        fmoduli_non_dir = fname_non_moduli[i]
        #load elements from fe
        Xread_non = np.loadtxt (fstruture_non_dir, dtype = float, delimiter = 'None')
        #load moduli from fe results
        #as long string
        Yread_non = np.genfromtxt (fmoduli_non_dir, dtype = str, delimiter = '\t')
        #split long string to string array
        ex_non = float(re.findall(r"\d+.\d+",Yread_non[0])[0])
        ey_non = float(re.findall(r"\d+.\d+",Yread_non[2])[1])
        ez_non = float(re.findall(r"\d+.\d+",Yread_non[4])[2])
        #string to float
        Yread_non = [ex_non, ey_non, ez_non]
        #average x, y, z
        Yread_p_non = np.array(Yread_non)
        #stack data
        Xread_non = torch.as_tensor(Xread_non)
        moduli_non = np.vstack((moduli_non,Yread_p_non))
        elems_nonsymm = np.vstack((elems_nonsymm, Xread_non))
        
    elems_3d_nonsymm = np.indices(strut_nonsymm_size)
    elems_3d_1d_nonsymm = elems_3d_nonsymm.reshape(3,-1)
    elems_3d_indexs_nonsymm = np.empty(shape = (len(fname_non_structure),\
                                        3,\
                                        np.size(elems_nonsymm_size)))
    elems_3d_data_nonsymm = np.zeros((len(fname_non_structure),\
                            strut_nonsymm_size[0],\
                            strut_nonsymm_size[1],\
                            strut_nonsymm_size[2]))

    for i in range (len(fname_non_structure)):    
        elems_3d_indexs_nonsymm[i,:,:] = np.vstack((elems_3d_1d_nonsymm[2][elems_nonsymm[i].astype(int)-1],\
                                 elems_3d_1d_nonsymm[1][elems_nonsymm[i].astype(int)-1],\
                                 elems_3d_1d_nonsymm[0][elems_nonsymm[i].astype(int)-1]))

        elems_3d_data_nonsymm[i][elems_3d_indexs_nonsymm[i][0].astype(int),\
                        elems_3d_indexs_nonsymm[i][1].astype(int),\
                        elems_3d_indexs_nonsymm[i][2].astype(int)] = 1

    elems_fnl_non = elems_3d_data_nonsymm.reshape(len(fname_non_structure),1,
                                    strut_size[0],
                                    strut_size[1],
                                    strut_size[2])
    
    return moduli_non, elems_fnl_non


In [5]:
# 3. read symm training dataset
def read_symm_data(fname_structure, fname_moduli):
    
    elems_symm  = np.empty(shape = (0, 7))
    elems_symm_size  = np.empty(shape = (1, 7))
    elems  = np.empty(shape = (0, 56))
    moduli = np.empty(shape = (0, 3))
    strut_symm_size = [3, 3, 3]
    strut_size = [6, 6, 6]
    
    for i in range (len(fname_structure)):
        fstruture_dir = fname_structure[i]
        fmoduli_dir = fname_moduli[i]
        #load elements from fe
        Xread = np.loadtxt (fstruture_dir, dtype = float, delimiter = 'None')
        #load moduli from fe results
        #as long string
        Yread = np.genfromtxt (fmoduli_dir, dtype = str, delimiter = '\t')
        #split long string to string array
        ex = float(re.findall(r"\d+.\d+",Yread[0])[0])
        ey = float(re.findall(r"\d+.\d+",Yread[2])[1])
        ez = float(re.findall(r"\d+.\d+",Yread[4])[2])
        #string to float
        Yread = [ex, ey, ez]
        #average x, y, z
        Yread_p = np.array(Yread)
        #stack data
        Xread = torch.as_tensor(Xread)
        moduli = np.vstack((moduli,Yread_p))
        elems_symm = np.vstack((elems_symm, Xread))

    elems_3d = np.indices(strut_symm_size)
    elems_3d_1d = elems_3d.reshape(3,-1)
    elems_3d_indexs = np.empty(shape = (len(fname_structure),\
                                        3,\
                                        np.size(elems_symm_size)))
    elems_3d_data = np.zeros((len(fname_structure),\
                            strut_symm_size[0],\
                            strut_symm_size[1],\
                            strut_symm_size[2]))

    for i in range (len(fname_structure)):
        elems_3d_indexs[i,:,:] = np.vstack((elems_3d_1d[2][elems_symm[i].astype(int)-1],\
                                 elems_3d_1d[1][elems_symm[i].astype(int)-1],\
                                 elems_3d_1d[0][elems_symm[i].astype(int)-1]))

        elems_3d_data[i][elems_3d_indexs[i][0].astype(int),\
                        elems_3d_indexs[i][1].astype(int),\
                        elems_3d_indexs[i][2].astype(int)] = 1

    elems_3d_data_symmed_1 = np.flip(elems_3d_data,axis=1)
    elems_3d_data_conc_1 = np.concatenate((elems_3d_data,
                                           elems_3d_data_symmed_1),axis = 1)
    elems_3d_data_symmed_2 = np.flip(elems_3d_data_conc_1,axis=2)
    elems_3d_data_conc_2 = np.concatenate((elems_3d_data_conc_1,
                                           elems_3d_data_symmed_2),axis = 2)
    elems_3d_data_symmed_3 = np.flip(elems_3d_data_conc_2,axis=3)
    elems_fnl_0 = np.concatenate((elems_3d_data_conc_2,elems_3d_data_symmed_3),axis = 3)
    elems_fnl = elems_fnl_0.reshape(len(fname_structure),1,
                                    strut_size[0],
                                    strut_size[1],
                                    strut_size[2])
    
    return moduli, elems_fnl

In [6]:
# 4. generated sample FEM moduli check

# import data
def moduli_check(fmoduli_fem_dir0):
#     fmoduli_fem_dir0 = r'/home/zhengyang/Data/trained0628'
    fname_m_fem = glob.glob(os.path.join(fmoduli_fem_dir0,'datagen_*.txt'))
    fname_m_fem.sort(key = lambda x:(int(re.split('_',x)[1]),
                                           int(re.split('_',x)[2]),
                                               int(re.split('_',x)[3])))
    f_count = 0
    moduli_fem = np.empty(shape = (0, 3))
    for i in fname_m_fem:
        f_count += 1
        Yread = np.loadtxt (i, dtype = str, delimiter = '\t')
        ex = float(re.findall(r"\d+.\d+",Yread[0])[0])
        ey = float(re.findall(r"\d+.\d+",Yread[2])[1])
        ez = float(re.findall(r"\d+.\d+",Yread[4])[2])
        Yread = [ex, ey, ez]
        Yread_p = np.array(Yread)
        moduli_fem = np.vstack((moduli_fem,Yread_p))
        
    return moduli_fem

In [7]:
# 5. generated sample binary structure load

def load_g_struc_b(fstruct_dir0, binary_lower):
#     fresults_dir0 = (r'/home/zhengyang/Data/generate_samples_binary_0706v2/')
    fname_struc = glob.glob(os.path.join(fstruc_dir0 ,'*.pt'))
    fname_struc.sort(key = lambda x:(int(re.split('_',x)[5]),
                                           int(re.split('_',x)[7]),
                                               int(re.split('_',x)[9])))
    
    g_struct_s = np.empty([len(fname_struc),8,2,6,6,6])
    sign = -1
    
    for i in fname_struc:
        sign += 1
        g_struct = torch.load(i)
        g_struct_s[sign,:,:,:,:,:] = g_struct.cpu()  > binary_lower
    
    return g_struct_s

In [8]:
# 6. generated sample continum structure load

def load_g_struc_c(fresults_dir0):
# fresults_dir0 = (r'/home/zhengyang/Data/generate_samples_binary_0706v2/')
# i = 5, j = 7, k =9;
    fname_results = glob.glob(os.path.join(fresults_dir0 ,'*.pt'))
    fname_results.sort(key = lambda x:(int(re.split('_',x)[5]),
                                           int(re.split('_',x)[7]),
                                               int(re.split('_',x)[9])))
#     fname_results.sort(key = lambda x:(int(re.split('_',x)[6])))
    g_struct_s = np.empty([len(fname_results),8,2,6,6,6])
    sign = -1
    
    for i in fname_results:
        sign += 1
        g_struct = torch.load(i)
        g_struct_s[sign,:,:,:,:,:] = g_struct.cpu()
        
    return g_struct_s

In [9]:
# 7. generated sample symmetry check
# input data type = np.array, input data shape = [len(), 8, 2, 6, 6, 6]

def symm_check_2c(g_struct_s):  
        
    symmz = g_struct_s[0,0,1,:,:,0:3]-np.flip(g_struct_s[0,0,1,:,:,3:6],axis = 4)
    symmy = g_struct_s[0,0,1,:,0:3,:]-np.flip(g_struct_s[0,0,1,:,3:6,:],axis = 3)
    symmx = g_struct_s[0,0,1,0:3,:,:]-np.flip(g_struct_s[0,0,1,3:6,:,:],axis = 2)
    
    # symmf = g_struct_s-np.flip(g_struct_s)
    symm_x = abs(symmx)
    symm_y = abs(symmy)
    symm_z = abs(symmz)

    sta_symm_x = np.sum(np.sum(np.sum(symm_x, axis = -1), axis = -1), axis = -1)
    sta_symm_y = np.sum(np.sum(np.sum(symm_y, axis = -1), axis = -1), axis = -1)
    sta_symm_z = np.sum(np.sum(np.sum(symm_z, axis = -1), axis = -1), axis = -1)

    sta_symm_x_mean = np.sum(sta_symm_x)/len(sta_symm_x)
    sta_symm_y_mean = np.sum(sta_symm_y)/len(sta_symm_y)
    sta_symm_z_mean = np.sum(sta_symm_z)/len(sta_symm_z)
    
    sta_symm_mean = np.array([sta_symm_x_mean, sta_symm_y_mean, sta_symm_z_mean])
    
    return sta_symm_x, sta_symm_y, sta_symm_z, sta_symm_x, sta_symm_mean

In [10]:
# anisotropy check


In [11]:
# generated sample fem element number

def get_fem_elems(fname_struc, vsave_dir, batch_size):
    f_n_c = 0
#     batch_size = 8
    for f_name in fname_g:
        f_n_c = f_n_c + 1
        for i in range(0,batch_size):
            g_struct = torch.load(f_name)
            f_n_cstr = str(f_n_c)
            iistr = str(i)
            g_struct = g_struct[i][1].cpu()
            gs_np = g_struct.numpy()
            gs_np_1d = gs_np.reshape(-1)
            gs_np_idx_re = np.argwhere(gs_np_1d > 1)
            gs_np_idx_re = gs_np_idx_re.reshape(-1) + 1
#             vsave_dir = (r'/home/zhengyang/Data/trained/elsel_'
#                          + f_n_cstr + '_' + iistr + '_' + '.txt')
            vsave_txt = open(vsave_dir, 'w')
            np.savetxt(vsave_dir,gs_np_idx_re,fmt="%d")
            vsave_txt.close()
    
    return gs_np_idx_re


In [13]:
# correlation coefficient check

def corr_coef_check():
    
    return